In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import plotly.express as px
import plotly.graph_objs as go
import nbformat
from sklearn.metrics import precision_recall_curve
nbformat.__version__

'5.8.0'

In [37]:
all_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\NLP_allData.csv")
user_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\complete_user_data.csv")
low = 'Low'
high = 'High'
user_data['cluster'] = user_data['cluster'].replace({0: low, 1: high})
user_data['custom_categorisation'] = user_data['custom_categorisation'].replace({0: low, 1: high})
column_names = {'language_diversity_index': 'Language Diversity Index', 'decade_diversity_index':'Decade Diversity Index','cluster': 'Clustered Diversity', 'mean_polarity': 'Polarity', 'mean_subjectivity':'Subjectivity',
       'mean_story_mentioned': 'Story Mentioned', 'mean_filmmaking_mentioned': 'Filmmaking Mentioned',
       'mean_adjectives_mentioned': 'Adjectives Used',
       'custom_categorisation': 'Polarized Diversity'}
user_data = user_data.rename(columns=column_names)
print(user_data.columns)
list = all_data.loc[all_data['adjectives_mentioned']==1]['review'].to_list()
len(list)
numerical_columns = ['Polarity', 'Subjectivity', 'Story Mentioned', 'Filmmaking Mentioned', 'Adjectives Used']
user_data = user_data.dropna(subset=numerical_columns)

# user_data

Index(['username', 'Language Diversity Index', 'Decade Diversity Index',
       'user_type', 'Clustered Diversity', 'Polarity', 'Subjectivity',
       'Story Mentioned', 'Filmmaking Mentioned', 'Adjectives Used',
       'mean_review_word_count', 'Polarized Diversity'],
      dtype='object')


In [38]:
user_data.columns
user_data_polarised = user_data.dropna(subset=['Polarized Diversity'])
user_data_polarised = user_data_polarised.iloc[::-1]

user_data_clustered = user_data.dropna(subset=['Clustered Diversity'])
binary_column = 'Clustered Diversity'
# new_df = user_data[user_data[binary_column] == low].sort_values('Decade Diversity Index').head(198)
# user_data_clustered = pd.concat([user_data.loc[user_data['Clustered Diversity']==high], new_df])
user_data_clustered = user_data_clustered.iloc[::-1]

# user_data_clustered

In [39]:
user_data_clustered[numerical_columns].describe()

,Polarity,Subjectivity,Story Mentioned,Filmmaking Mentioned,Adjectives Used
count,726.000000,726.000000,726.000000,726.000000,726.000000
mean,0.125192,0.410443,0.143550,0.134976,0.210738
std,0.092461,0.147372,0.193550,0.181382,0.216025
min,-0.142726,0.000000,0.000000,0.000000,0.000000
25%,0.057809,0.348983,0.017703,0.000000,0.052632
50%,0.120176,0.446663,0.065217,0.060000,0.142857
75%,0.181761,0.511864,0.196078,0.198913,0.295120
max,0.574574,0.745946,1.000000,1.000000,1.000000


CLUSTERED ANALYSIS

In [40]:
fig = px.scatter(user_data_clustered, x='Decade Diversity Index', y='Language Diversity Index', color='Clustered Diversity', width=700, height=500)
fig.show()

In [41]:
fig = px.box(user_data_clustered.melt(id_vars=['Clustered Diversity'], value_vars=numerical_columns), x='variable', y='value', color='Clustered Diversity')

fig.show()

In [42]:
user_data_clustered.loc[user_data_clustered[binary_column]==low][numerical_columns].describe()

,Polarity,Subjectivity,Story Mentioned,Filmmaking Mentioned,Adjectives Used
count,528.000000,528.000000,528.000000,528.000000,528.000000
mean,0.127826,0.406341,0.117242,0.110677,0.185765
std,0.095937,0.145521,0.163752,0.158211,0.192419
min,-0.119792,0.000000,0.000000,0.000000,0.000000
25%,0.057506,0.342179,0.006550,0.000000,0.045455
50%,0.124440,0.440153,0.060000,0.044949,0.130435
75%,0.184814,0.506674,0.138332,0.142857,0.260217
max,0.574574,0.745946,0.940000,1.000000,0.980000


In [43]:
user_data_clustered.loc[user_data_clustered[binary_column]==high][numerical_columns].describe()

,Polarity,Subjectivity,Story Mentioned,Filmmaking Mentioned,Adjectives Used
count,198.000000,198.000000,198.000000,198.000000,198.000000
mean,0.118167,0.421382,0.213705,0.199773,0.277333
std,0.082294,0.152030,0.243582,0.219803,0.258007
min,-0.142726,0.000000,0.000000,0.000000,0.000000
25%,0.067380,0.368907,0.022118,0.020435,0.076923
50%,0.116878,0.472559,0.140000,0.132012,0.208818
75%,0.169913,0.520032,0.321488,0.300000,0.398958
max,0.415811,0.685798,1.000000,0.980000,1.000000


In [44]:
results = []
for column in numerical_columns:
    group1 = user_data_clustered[user_data_clustered[binary_column] == low][column]
    group2 = user_data_clustered[user_data_clustered[binary_column] == high][column]
    t_statistic, p_value = ttest_ind(group1, group2)
    results.append([column, t_statistic, p_value])

df = pd.DataFrame(results, columns=['Dependent Variable', 'T-Statistic', 'P-Value'])
df

,Dependent Variable,T-Statistic,P-Value
0,Polarity,1.254099,2.102107e-01
1,Subjectivity,-1.225179,2.209060e-01
2,Story Mentioned,-6.129656,1.446576e-09
3,Filmmaking Mentioned,-6.036805,2.510187e-09
4,Adjectives Used,-5.176231,2.935954e-07


POLARISED ANALYSIS

In [45]:
binary_column = 'Polarized Diversity'

In [46]:
user_data_polarised[numerical_columns].describe()

,Polarity,Subjectivity,Story Mentioned,Filmmaking Mentioned,Adjectives Used
count,73.000000,73.000000,73.000000,73.000000,73.000000
mean,0.109714,0.407792,0.192852,0.164017,0.251150
std,0.083605,0.153677,0.232038,0.182360,0.253441
min,-0.142726,0.000000,0.000000,0.000000,0.000000
25%,0.047131,0.320738,0.015873,0.000000,0.040816
50%,0.114035,0.451304,0.083333,0.120000,0.166667
75%,0.147728,0.522756,0.326531,0.230769,0.363636
max,0.360133,0.693023,0.900000,0.717949,0.934783


In [48]:
fig = px.scatter(user_data_polarised, x='Decade Diversity Index', y='Language Diversity Index', color='Polarized Diversity', width=700, height=500)
fig.show()

In [20]:
fig = px.box(user_data_polarised.melt(id_vars=['Polarized Diversity'], value_vars=numerical_columns), x='variable', y='value', color='Polarized Diversity')

fig.show()

In [21]:
user_data_polarised.loc[user_data_polarised[binary_column]==low][numerical_columns].describe()

,Polarity,Subjectivity,Story Mentioned,Filmmaking Mentioned,Adjectives Used
count,33.000000,33.000000,33.000000,33.000000,33.000000
mean,0.104716,0.363259,0.099651,0.092569,0.170715
std,0.097694,0.175181,0.181302,0.147703,0.204039
min,-0.057669,0.000000,0.000000,0.000000,0.000000
25%,0.033957,0.292992,0.000000,0.000000,0.036364
50%,0.090206,0.417953,0.040000,0.020408,0.093750
75%,0.145960,0.480178,0.102041,0.137931,0.280000
max,0.360133,0.693023,0.900000,0.680000,0.900000


In [22]:
user_data_polarised.loc[user_data_polarised[binary_column]==high][numerical_columns].describe()

,Polarity,Subjectivity,Story Mentioned,Filmmaking Mentioned,Adjectives Used
count,40.000000,40.000000,40.000000,40.000000,40.000000
mean,0.113838,0.444533,0.269742,0.222961,0.317509
std,0.070955,0.123923,0.242931,0.188709,0.272823
min,-0.142726,0.000000,0.000000,0.000000,0.000000
25%,0.085993,0.380875,0.077442,0.087054,0.130084
50%,0.117347,0.490232,0.214583,0.184659,0.246364
75%,0.163710,0.525601,0.421218,0.354706,0.481146
max,0.226629,0.595920,0.871795,0.717949,0.934783


In [23]:
results = []
for column in numerical_columns:
    group1 = user_data_polarised[user_data_polarised[binary_column] == low][column]
    group2 = user_data_polarised[user_data_polarised[binary_column] == high][column]
    t_statistic, p_value = ttest_ind(group1, group2)
    results.append([column, t_statistic, p_value])

df = pd.DataFrame(results, columns=['Dependent Variable', 'T-Statistic', 'P-Value'])
df

,Dependent Variable,T-Statistic,P-Value
0,Polarity,-0.461418,0.645909
1,Subjectivity,-2.316052,0.023447
2,Story Mentioned,-3.328062,0.001388
3,Filmmaking Mentioned,-3.234074,0.001853
4,Adjectives Used,-2.555821,0.012736
